In [1]:
import sys
import numpy as np
import pandas as pd

sys.path.append('/home/charon/project')

from shallowmind.api.infer import prepare_inference

In [2]:
ckpt = '/data2/charon/LCI/ci_lingam-n_datasets=100-n_samples=500_10000-model=resnet50-embedding=128-lr=0.1-epoch=100/ckpts/exp_name=ci_lingam-n_datasets=100-n_samples=500_10000-model=resnet50-embedding=128-lr=0.1-epoch=100-cfg=ci_lingam-n_datasets=100-n_samples=500_10000-model=resnet50-embedding=128-bs=128-seed=42-val_mean_squared_error=0.0803.ckpt'
cfg = '/data2/charon/LCI/ci_lingam-n_datasets=100-n_samples=500_10000-model=resnet50-embedding=128-lr=0.1-epoch=100/ci_lingam-n_datasets=100-n_samples=500_10000-model=resnet50-embedding=128.py'

di, mi = prepare_inference(cfg, ckpt)
mi = mi.cuda()

In [3]:
# load flu dataset
flu_df = pd.read_stata('/home/charon/project/learning-ivs/data/flu_clean.dta')

In [26]:
import torch
from copy import deepcopy
inp = flu_df[['assign', 'receive', 'outcome']]
# inp = flu_df[['assign', 'receive', 'outcome']]
inp['n_samples'] = len(inp)
inp = torch.from_numpy(inp.to_numpy()).to(torch.float32).cuda()


/tmp/ipykernel_284251/3598825097.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inp['n_samples'] = len(inp)


In [27]:
inp.shape

torch.Size([2861, 4])

In [28]:
seed = 42
def get_samples(inp, i, n_samples=500):
    np.random.seed(seed)
    if (i+1) * n_samples > inp.shape[0]:
        return inp[np.random.choice(inp.shape[0], n_samples, replace=True), :]
    else:
        return inp[i*n_samples:(i+1)*n_samples, :]

inp = torch.stack([get_samples(inp, i) for i in range(len(inp)//500+1)], dim=0)
inp.shape

torch.Size([6, 500, 4])

In [29]:
mi(inp).mean()

tensor(-0.2197, device='cuda:0', grad_fn=<MeanBackward0>)